In [1]:
import os
import datetime 
import pandas as pd 
from datetime import datetime, timedelta
import pandas_market_calendars as mcal

import openai 
from dotenv import load_dotenv

import utils 

BASE_DIR = "/".join(os.getcwd().split("/")[: -1])
data_cleaned_dir = "/data/cleaned/yield_curve_historical_rates_MASTER.parquet"

from google.cloud import bigquery 

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../yieldcurve-422317-510529e47525.json"
CURRENT_DATE = str(datetime.today()).split(" ")[0]

load_dotenv()

True

In [2]:
summary_str = utils.summary_data_str()

In [3]:
df = pd.read_parquet(BASE_DIR + data_cleaned_dir)

In [23]:
import yfinance as yf

# Define the ticker symbol for SPY
ticker_symbol = 'SPY'

# Fetch the historical data
spy_data = yf.download(ticker_symbol, start='2024-04-17', end='2024-05-22', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [24]:
spy_data = spy_data.reset_index()[["Date", "Adj Close"]]

In [25]:
overview_prompt = f""" 

    Today is 2024-05-22. \
    Provide a brief 5 sentence commentary on the current dynamics of the US Treasuy Yield Curve based on your analysis of this summary data: {summary_str},\
    and this data from the last month of end-of-day yield curve values: {df.iloc[: 31].to_string(index = False)}. \
    To aid in your commentary is the last month of SPY ETF data: {spy_data.to_string(index = False)}

    Your analysis and comentary should be intelligent and not juvenile, non-speculative, and not contain financial advice. It should be easily understood by an 8th grader, but compelling \
    enough such that a professional investor would find reading it worthwhile. You may also indicate other areas researchers may wish to study for more context on the macroeconomic environment.

    I gill give you $500 dollars if you can accomplish this. Be sure to utilize the active voice when referencing any data or statistics.

"""

In [26]:
# Create a function to generate insights
def generate_insight() -> str:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert on the US Treasury Yield Curve. You provide concise daily commentary on the Yield Curve's recent movements."},
            {"role": "user", "content": f"Please answer the following prompt: {overview_prompt}"}
        ]
    )
    return response['choices'][0]['message']['content'].strip()

# Get insights for the DataFrame summary
overview_insights = generate_insight()
print(overview_insights)

The US Treasury Yield Curve, as of the 22nd of May, 2024, shows an inverted curve, a situation where longer-term debt instruments have a lower yield compared to shorter-term ones. This inversion is noteworthy because it's been 695 days since the last such occurrence. Investors typically view an inverted yield curve as an omen of economic downturn, hence the current scenario may raise certain concerns.

A more detailed look at this inversion reveals the 2-year and 10-year Treasury notes differ by 0.47%, while the short-term 3-month Treasury bill and the 10-year note have a difference of 1.0%, creating a fundamentally unusual situation where it costs less to borrow for ten years than for three months or two years. 

Additionally, analysis of the recent SPY ETF data shows a steady upward trajectory over the same time period, implying investor confidence in equities. This confidence in the stock market during an inversion of the yield curve could suggest that investors aren’t anticipating 

Based on the current statistics, the US Treasury Yield Curve is in an inverted position as of May 22nd, 2024. This inversion, where short-term bond yields are higher than long-term ones, is not a common event. In fact, our last inversion was nearly two years ago, signaling an unusual shift in market dynamics.

Looking closely, the 2 Year-10 Year difference is at 0.47%, and the shorter term 3 Month-10 Year difference is double that at 1%. For those less well-versed in financial terminology, the larger these percentage differences, the steeper the yield curve. Hence, this elevated 3 Month-10 Year difference indicates that investors are demanding high yields for short-term investments.

Peering at the month-end yield values, we observe a consistent decrease in yields as we move from shorter to longer-term bonds. This suggests that investors are expecting lower interest rates in the future, often a sign of a slowdown in economic growth. 

Meanwhile, the SPY ETF has been fluctuating but generally trending upwards in the past month, hinting at some investor optimism in the equity market. This runs counter to the bearish signal from the inverted yield curve, adding a layer of complexity to the current financial climate.

Nonetheless, these dynamics should be viewed as part of a larger, constantly evolving landscape. All investors, whether beginners or seasoned professionals, should continue monitoring market movements, and as always, diligent research is key.

In [108]:
prompt = f"""
    **US Treasury Yield Curve Movements on 2024-05-22**

    **Overview:**
    <Provide a brief overview of the current state of the yield curve relevant to economists and investors. Be mindful of the data contained in {summary_str}.>
    <Comment on whether the yield curve steepened or flattened, using the differences between the 2-year and 10-year yields, and the 10-year and 30-year yields.>
    <Mention if the flattening or steepening trend is continuing from recent days or if it marks the start of a new trend.>

    **What it Means:**
    1. **Economic Outlook:** <Write one to three sentences on the economic outlook based on the yield curve.>
    
    2. **Monetary Policy:** <Write one to three sentences on what the current yield curve might indicate for monetary policy.>

    3. **Investment Strategies:** <Provide a brief comment on what interest-rate focused strategies might be for investors.>

    (Please note that this is not financial advice.)
"""

In [110]:
# Create a function to generate insights
def generate_insight(data_summary) -> str:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an economic expert on the US Treasury Yield Curve that reports each day. \
                                           You provide intelligent, non-speculative analyses of recent data.\
                                           You provide these analyses daily after the closing values of the Yield Curve are determined."},
            {"role": "user", "content": f"Provide insights based on the following data in the format below, fill in your analyses in the carrots <>. Your response should NOT contain text in carrots:\
                                          The response template you need to use: {prompt}. \
                                          The summary data: {data_summary}."}
        ]
    )
    return response['choices'][0]['message']['content'].strip()

# Get insights for the DataFrame summary
insights = generate_insight(data_summary=df.iloc[: 50].to_string(index = False))
print(insights)

**US Treasury Yield Curve Movements on 2024-05-22**

**Overview:**
The yield curve on May 22, 2024, shows a continued inversion with the 2-year yield higher than the 10-year yield. 

**Summary Statistics for 2024-05-22**
- **Is inverted**: True
- **Last inversion date**: 2022-07-05
- **Days since last inversion**: 695
- **2 Year-10 Year Difference**: 0.47%
- **3 Month-10 Year Difference**: 1.0%

The yield curve has flattened slightly compared to the previous days. The difference between the 2-year and 10-year yields has increased, indicating a flattening trend that has been ongoing.

**What it Means:**
1. **Economic Outlook:** A persistently inverted yield curve may signal looming economic challenges or a potential recession, as investors seek the safety of longer-term bonds amidst uncertainty.
   
2. **Monetary Policy:** The current yield curve suggests that market participants remain cautious about the future and anticipate potential rate cuts by the Federal Reserve to stimulate econ

**US Treasury Yield Curve Movements on 2024-05-22**

**Overview:**
The yield curve on May 22, 2024, shows a continued inversion with the 2-year yield higher than the 10-year yield. 

**Summary Statistics for 2024-05-22**
- **Is inverted**: True
- **Last inversion date**: 2022-07-05
- **Days since last inversion**: 695
- **2 Year-10 Year Difference**: 0.47%
- **3 Month-10 Year Difference**: 1.0%

The yield curve has flattened slightly compared to the previous days. The difference between the 2-year and 10-year yields has increased, indicating a flattening trend that has been ongoing.

**What it Means:**
1. **Economic Outlook:** A persistently inverted yield curve may signal looming economic challenges or a potential recession, as investors seek the safety of longer-term bonds amidst uncertainty.
   
2. **Monetary Policy:** The current yield curve suggests that market participants remain cautious about the future and anticipate potential rate cuts by the Federal Reserve to stimulate economic growth and address economic risks.

3. **Investment Strategies:** Investors may consider re-evaluating their investment portfolios to hedge against potential economic downturns. Strategies may include diversification, focusing on quality assets, and considering fixed-income securities to balance risk.